In [3]:
import random
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
num_classes = 1
batch_size = 1
sample_size = 1
sequence_length = 1
learning_rate = 0.01
hidden_layer_size = 10


def norm_array(array):
    mean = np.mean(array)
    stddev = np.sqrt(np.sum(np.square(array - mean)) / len(array - 1))
    normalized_array = ((array - mean) / stddev)
    return normalized_array, mean.reshape(batch_size), stddev.reshape(batch_size)

def norm_test(array, mean, stddev):
    return (array - mean) / stddev

def test_decoder(array, mean, stddev):
    return (array * stddev) + mean

def get_linear_layer(vector):
    return(tf.matmul(vector, Wl) + bl)
def train_test_split(X, y, train_size):
    X_train = X[:int(len(X) // (1 / train_size))]
    y_train = y[:int(len(y) // (1 / train_size))]
    X_test = X[int(len(X) // (1 / train_size)):]
    y_test = y[int(len(y) // (1 / train_size)):]
    return X_train, X_test, y_train, y_test

def data_labels(data, sample_size, batch_size):
    labels = data.iloc[:sample_size, 2:]
    data = data.iloc[:sample_size, 1:]
    indeces = np.random.choice(len(data), size=batch_size, replace=False)
    batch_x = np.array(data.loc[indeces[0]][data.loc[indeces[0]][:] > 0][:-1])
    batch_y = np.array(labels.loc[indeces[0]][labels.loc[indeces[0]][:] > 0])
    return (batch_x, batch_y)


def cell_list(num_LSTM_layers):
    cell_list = []
    for i in range(num_LSTM_layers):
        cell_list.append(tf.contrib.rnn.BasicLSTMCell(hidden_layer_size, forget_bias=1.0))
    return cell_list

def mse_diff(mse_batch_old, mse_batch_new):
    return ((mse_batch_old - mse_batch_new)**2)**(1/2)


In [5]:
data_monthly = pd.read_csv(r"C:\M4datasets\Monthly-train.csv")
data_month, labels_month = data_labels(data_monthly, sample_size = 1, batch_size = 1)
data_month = data_month[:90]
labels_month = labels_month[:90]

In [6]:
num_layers = [1, 2, 3, 4, 5]
num_hidden = [1, 2, 5, 10, 15, 20, 30, 50, 75, 100, 125, 150, 200]
param_list = [(x, y) for x in num_layers for y in num_hidden]

In [8]:




stop_list = []
for param in param_list:
    num_LSTM_layers = param[0]
    hidden_layer_size = param[1]
    tf.reset_default_graph()
    with tf.name_scope("Training_data"):
        inputs = tf.placeholder(tf.float32,shape=[batch_size, None, sequence_length], name='inputs')
        y = tf.placeholder(tf.float32, shape=[batch_size], name='inputs')

    with tf.name_scope('Linear_weights_and_biases'):
        Wl = tf.Variable(tf.truncated_normal([hidden_layer_size, num_classes], mean = 0, stddev = 0.01))
        bl = tf.Variable(tf.truncated_normal([num_classes], mean = 0, stddev = 0.1))

    with tf.name_scope('Means_and_standard_deviations'):
        mean = tf.placeholder(tf.float32, shape = [batch_size])
        stddev = tf.placeholder(tf.float32, shape = [batch_size])

    with tf.variable_scope('LSTM_cell'):  
        cell = tf.contrib.rnn.MultiRNNCell(cells=cell_list(num_LSTM_layers), state_is_tuple=True)
        outputs, states = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)


    with tf.variable_scope('Batch_outputs'):
        final_outputs = tf.placeholder

    linear_output = get_linear_layer(outputs[0])
    final_output = test_decoder(linear_output, mean, stddev)[-1]
    mse = tf.reduce_mean(tf.squared_difference(final_output, y[0]))
    train_step = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(mse)

    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer())

    mse_list = []
    mse_batch_old = 1e-10
    for i in range(2):
        batch_x, batch_y = data_month, labels_month
        data_train, data_test, labels_train, labels_test = train_test_split(batch_x, batch_y, train_size = 0.8)

        normed_data_train, mean_batch, stddev_batch = norm_array(data_train)

        normed_data_train = normed_data_train.reshape(batch_size, len(data_train), sequence_length)

        label = labels_train[-1].reshape(batch_size)
        sess.run(train_step, feed_dict = {inputs : normed_data_train,\
                                                y : label,  mean : mean_batch, stddev : stddev_batch})
        mse_batch = sess.run(mse, feed_dict = {inputs : normed_data_train,\
                                                y : label,  mean : mean_batch, stddev : stddev_batch})
        for j in range(len(data_test)):
            data_train =  np.append(data_train, data_test[j])
            labels_train = np.append(labels_train, labels_test[j])
            normed_data_train, mean_batch, stddev_batch = norm_array(data_train)
            normed_data_train = normed_data_train.reshape(batch_size, len(data_train), sequence_length)

            label = labels_train[-1].reshape(batch_size)
            sess.run(train_step, feed_dict = {inputs : normed_data_train, y : label, mean : mean_batch, stddev : stddev_batch})
            mse_batch += sess.run(mse, feed_dict = {inputs : normed_data_train,\
                                                y : label,  mean : mean_batch, stddev : stddev_batch})
        mse_batch_new = mse_batch / len(data_test)
        if i % 10 == 0:
            print("Criteria:",mse_diff(mse_batch_old, mse_batch_new), "Iter:", i)
        if mse_batch_new <= 1:
            stop_list.append(i)
            mse_batch_old = mse_batch_new
            break
        if mse_diff(mse_batch_old, mse_batch_new) < 0.001 and mse_batch_new > 1 :
            stop_list.append(2000)
            mse_batch_old = mse_batch_new
            break
        mse_batch_old = mse_batch_new
        if i == 1999:
            stop_list.append(2000)
            break
        if mse_batch_old < 5000:
            learning_rate == learning_rate / ((i+1) * 10000)
       
    print(param, len(stop_list), mse_batch_old)

C:\1\envs\tensorflow\lib\site-packages\tensorflow\python\client\session.py:1711: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Criteria: 2562456.0 Iter: 0
(1, 1) [] 0 1674361.6666666667
Criteria: 0.0 Iter: 1
Criteria: 2236004.222222222 Iter: 0
(1, 2) [] 0 1343173.2222222222
Criteria: 0.0 Iter: 1
Criteria: 1885492.888888889 Iter: 0
(1, 5) [] 0 741012.7777777778
Criteria: 0.0 Iter: 1
Criteria: 1621531.111111111 Iter: 0
(1, 10) [] 0 563277.8888888889
Criteria: 0.0 Iter: 1
Criteria: 1609398.888888889 Iter: 0
(1, 15) [] 0 536217.1666666666
Criteria: 0.0 Iter: 1
Criteria: 1721793.111111111 Iter: 0
(1, 20) [] 0 565455.8333333334
Criteria: 0.0 Iter: 1
Criteria: 1124793.4444444445 Iter: 0
(1, 30) [] 0 913679.1111111111
Criteria: 0.0 Iter: 1
Criteria: 821834.111111111 Iter: 0
(1, 50) [] 0 1161558.6666666667
Criteria: 0.0 Iter: 1


KeyboardInterrupt: 